In [ ]:
print("Ok")

: 

In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone as PineconeStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers



In [ ]:
PINCONE_API_KEY = ""


In [4]:
# Extract data from pdf

def load_pdf(data):
   loader =  DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
   documents = loader.load()
   return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
#extracted_data


In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("lenght of my chunks: ", len(text_chunks))

lenght of my chunks:  5859


In [9]:
#text_chunks



In [10]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
from sentence_transformers import SentenceTransformer
from huggingface_hub import hf_hub_download
print("✅ Import works!")

c:\Users\maury\anaconda3\envs\mechatbot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


✅ Import works!


In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\maury\AppData\Local\Temp\ipykernel_23600\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("legth", len(query_result))

legth 384


In [15]:
#query_result




In [16]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINCONE_API_KEY)
index_name = "medical-chatbot"
index = pc.Index(index_name)
# if index_name not in pc.list_indexes().names():
#     pc.create_index(
#         name=index_name,
#         dimension=384,  # depends on your embedding model
#         metric="cosine",
#          spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#          )
#     )

# PineconeStore.from_texts([t.page_content for t in text_chunks], embedding = embeddings, index_name=index, namespace="default")

In [17]:
index

In [18]:
# vectorstore = PineconeStore.from_texts(
#     [t.page_content for t in text_chunks],
#     embedding=embeddings,
#     index_name=index,
#     namespace="default"
# )

In [ ]:
index = pc.Index(index_name)
vectorstore = PineconeStore(index, embeddings.embed_query, "text")
vectorstore.add_texts([t.page_content for t in text_chunks])

C:\Users\maury\AppData\Local\Temp\ipykernel_23600\1718670387.py:2: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  vectorstore = PineconeStore(index, embeddings.embed_query, "text")
c:\Users\maury\anaconda3\envs\mechatbot\lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [20]:
# k=2 will give top 2 result
results = vectorstore.similarity_search("What is Allergies?", k=2)
print(results)
# for r in results:
#     print(r.page_content)


[Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of')]


In [21]:
prompt_template='''
Use the following pieces of information to answer the user's question.
If you dont know the answer, just say that you dont know, do'nt try to make up an answer.

Context:{context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
'''

In [22]:
PROMT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMT}

In [23]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature': 0.8})

In [24]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
# while True:
#     user_input = input(f"Input Prompt:")
#     result = qa({"query": user_input})
#     print("Response : ", result["result"])
    

C:\Users\maury\AppData\Local\Temp\ipykernel_5284\1781939318.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa({"query": user_input})


Response :  The incubation period for HIV, the virus that causes AIDS, is typically around 2-6 weeks after exposure to the virus. However, it can take up to several months or even years for symptoms to appear.
Response :  Acne is a common skin condition characterized by inflamed or non-inflamed bumps on the face, chest, back, and other areas of the body. It is caused by a combination of factors such as hormonal changes, excess oil production, clogged pores, and bacterial infections. Acne can lead to emotional distress, social isolation, and reduced self-esteem if left untreated. Treatment options include topical creams and gels, oral medications, and lifestyle modifications such as regular exercise and a healthy diet.


In [25]:
user_input = input(f"Input Prompt:")
result = qa({"query": user_input})
print("Response : ", result["result"])

C:\Users\maury\AppData\Local\Temp\ipykernel_23600\3453996880.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa({"query": user_input})


Response :  Allergy is an overreaction of the immune system to a substance that is typically harmless but the body perceives as a threat. This can lead to a variety of symptoms including sneezing, congestion, runny nose, itchy eyes, and skin rashes.
